In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os

In [2]:
#data = pd.read_excel("Final_Data 5.xlsx")

In [3]:
'''x = data.drop('PM2.5', axis=1)
x = x.drop(['time', 'latitude', 'longitude', 'Abr'], axis=1)
#x = x.drop('Stations', axis=1)
#x = x.drop('longitude_c', axis=1)
#x = x.drop('latitude_c', axis=1)
y = data['PM2.5']'''

"x = data.drop('PM2.5', axis=1)\nx = x.drop(['time', 'latitude', 'longitude', 'Abr'], axis=1)\n#x = x.drop('Stations', axis=1)\n#x = x.drop('longitude_c', axis=1)\n#x = x.drop('latitude_c', axis=1)\ny = data['PM2.5']"

In [4]:
#x_cols = x.columns

In [5]:
folder = "./Visibility nc/"
paths = os.listdir(folder)
vis = [xr.open_dataset(folder + path) for path in paths]

folder = "./MAIAC AOD gap filled nc/"
paths = os.listdir(folder)
aod = [xr.open_dataset(folder + path) for path in paths]

folder = "./Era5 Land Assimilated/"
paths = sorted(os.listdir(folder))
era5 = [xr.open_dataset(folder + path) for path in paths]

folder = "./Roads nc/"
paths = sorted(os.listdir(folder))
roads = [xr.open_dataset(folder + path) for path in paths]

folder = "./Vegetation nc/"
paths = sorted(os.listdir(folder))
veg = [xr.open_dataset(folder + path).drop('EVI') for path in paths]

c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
for i in range(len(aod)):
    aod[i]['latitude'] = np.round(aod[i]['latitude'].values,2)
    aod[i]['longitude'] = np.round(aod[i]['longitude'].values,2)

for i in range(len(vis)):
    vis[i]['latitude'] = np.round(vis[i]['latitude'].values,2)
    vis[i]['longitude'] = np.round(vis[i]['longitude'].values,2)

for i in range(len(era5)):
    era5[i]['latitude'] = np.round(era5[i]['latitude'].values,2)
    era5[i]['longitude'] = np.round(era5[i]['longitude'].values,2)

for i in range(len(roads)):
    roads[i]['latitude'] = np.round(roads[i]['latitude'].values,2)
    roads[i]['longitude'] = np.round(roads[i]['longitude'].values,2)

for i in range(len(veg)):
    veg[i]['latitude'] = np.round(veg[i]['latitude'].values,2)
    veg[i]['longitude'] = np.round(veg[i]['longitude'].values,2)

In [7]:
nc_data = xr.merge([xr.concat(vis, dim='time'),
                    xr.concat(aod, dim='time'),
                    xr.concat(era5, dim='time'),
                    xr.concat(roads, dim='time'),
                    xr.concat(veg, dim="time")])

In [8]:
del aod, era5, vis

In [9]:
for var in list(nc_data.data_vars):
    nc_data[var].values = np.round(nc_data[var].values, 16).astype(np.float16)

In [10]:
nc_df1 = nc_data.to_dataframe().reset_index()

In [11]:
nc_df = nc_df1.copy()

In [12]:
del nc_data, nc_df1

In [13]:
nc_df.sort_values(['time', 'latitude', 'longitude'], ascending=[True, False, True], inplace=True)
nc_df.reset_index(inplace=True, drop=True)

In [14]:
def time_lag_features_1(in_df, col_name, lag_steps):
    lag = len(in_df[in_df['time'] == in_df['time'].unique()[0]])
    for i in range(lag_steps):
        in_df[col_name + '_' + f'lag{i+1}'] = in_df[col_name].shift(lag * (i+1))
        in_df[col_name + '_' + f'lag-{i+1}'] = in_df[col_name].shift(lag * -(i+1))
    return in_df

In [15]:
def time_lag_features_2(in_df, col_name, shift_list):
    len_lons = len(in_df['longitude'].unique())
    for i in shift_list:
        in_df[col_name + '_' + f'u{i}'] = in_df[col_name].shift(len_lons * (i))
        in_df[col_name + '_' + f'lw{i}'] = in_df[col_name].shift(-len_lons * (i))
        in_df[col_name + '_' + f'le{i}'] = in_df[col_name].shift((i))
        in_df[col_name + '_' + f'r{i}'] = in_df[col_name].shift(-(i))
        in_df[col_name + '_' + f'u-le{i}'] = in_df[col_name].shift(len_lons * (i) + (i))
        in_df[col_name + '_' + f'lw-r{i}'] = in_df[col_name].shift(-len_lons * (i) - (i))
        in_df[col_name + '_' + f'u-r{i}'] = in_df[col_name].shift(len_lons * (i) - (i))
        in_df[col_name + '_' + f'lw-le{i}'] = in_df[col_name].shift(-len_lons * (i) + (i))
    return in_df

In [16]:
s_lis_aod = [1, 2, 3, 4, 5, 6, 7, 8]
cols_aod = ['filled_AOD', 'filled_AOD_lag1', 'filled_AOD_lag-1', 
            'filled_AOD_lag2', 'filled_AOD_lag-2', 'filled_AOD_lag3', 'filled_AOD_lag-3',
            'filled_AOD_lag4', 'filled_AOD_lag-4', 'filled_AOD_lag5',
       'filled_AOD_lag-5']

In [17]:
s_lis_era5 = [10, 20, 30, 40]
cols_era5 = ['u10', 'v10', 'u10_lag1', 'v10_lag1']

In [18]:
nc_df = time_lag_features_1(nc_df, 'filled_AOD', 7)
nc_df = time_lag_features_1(nc_df, 'u10', 2)
nc_df = time_lag_features_1(nc_df, 'v10', 2)

for col in cols_aod:
    nc_df = time_lag_features_2(nc_df, col, s_lis_aod)
for col in cols_era5:
    nc_df = time_lag_features_2(nc_df, col, s_lis_era5)

nc_df['Day_of_Week'] = nc_df['time'].dt.day_of_week
nc_df['Day_of_Year'] = nc_df['time'].dt.day_of_year
nc_df['Month'] = nc_df['time'].dt.month
nc_df['Year'] = nc_df['time'].dt.year



C:\Users\user\AppData\Local\Temp\ipykernel_8336\798970617.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  in_df[col_name + '_' + f'lw-r{i}'] = in_df[col_name].shift(-len_lons * (i) - (i))
C:\Users\user\AppData\Local\Temp\ipykernel_8336\798970617.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  in_df[col_name + '_' + f'u-r{i}'] = in_df[col_name].shift(len_lons * (i) - (i))
C:\Users\user\AppData\Local\Temp\ipykernel_8336\798970617.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

In [19]:
del col, cols_aod, cols_era5, folder, i, paths, roads, s_lis_aod, s_lis_era5, var, veg

In [20]:
1523-15

1508

In [22]:
#nc_df.set_index(['time', 'latitude', 'longitude'], inplace=True, drop=True)

splits = np.array_split(nc_df.index, 1523)

for i in range(len(splits)):
    if (i > 1090) & (i < 1508):
        sp = nc_df.loc[splits[i]]
        sp.set_index(['time', 'latitude', 'longitude'], drop=True
                    ).astype({col: np.float32 for col in sp.select_dtypes(include=['float16']
                                                                        ).columns}
                                                                        ).to_xarray().to_netcdf(f'./dss/ds_{i}.nc')
    
del nc_df
